## **Mapeando datos**

➡️ Apuntar

🗒️ **Observaciones:**

⚠️ **Alertas:**

### **1. Importamos las respectivas librerías**

In [ ]:
import pandas as pd
import zipfile
from pathlib import Path
import shutil
import folium
import sqlite3

### **2. Descomprimimos los archivos correspondientes**

➡️ Creamos una variable path que nos ayude a indentificar donde estan todos los archivos zip.

In [ ]:
raw_path = '/home/npalaciosv/Catedra/Geoanalitycs/src/Data/Raw/'
silver_path = '/home/npalaciosv/Catedra/Geoanalitycs/src/Data/Silver/'

data_files = Path(raw_path)

➡️ Necesitamos recorrer la carpeta ya que habra mas de un archivo zip, luego vamos a descomprimirlos, almacenarlos en un dataframe y juntarlos todos. Usamos la libreria Path.

In [ ]:
for file in data_files.rglob('*'):
    new_name = file.name.replace(" ","").replace("_","")
    new_path = file.with_name(new_name)
    file.rename(new_path)
    with zipfile.ZipFile(f'{raw_path+file.name}','r') as extraction:
        extraction.extractall(silver_path)

➡️ Eliminamos los archivos que tienen un formato diferente a csv.

🗒️ **Observaciones:** La librería path tambien es útil en este caso.

In [ ]:
silver_files = Path(silver_path)
for file in silver_files.iterdir():
    # Verificar si es un archivo y no termina en '.csv'
    if file.is_file() and file.suffix != '.csv':
        # Eliminar el archivo
        file.unlink()
        print(f'Eliminado: {file}')

⚠️ **Alertas:** Algunos archivos se descomprimen dentro de una carpeta, por lo tanto nos toca moverlos hacia el directorio Silver

In [ ]:
for file in silver_files.iterdir():
    if file.is_file():
        pass
    else:
        subfolder = silver_files.joinpath(file.name)
        for subfile in subfolder.glob('*'):
            if subfile.suffix == '.csv':
                subfile.replace(silver_path+"/"+subfile.name)
                print(f'Archivo: {subfile.name} movido exitosamente')

        shutil.rmtree(subfolder)
        print(f'Carpeta: {subfolder.name} elminada')

### **3. Unificamos la informacion que necesitamos**

➡️ Creamos un dataframe vacio y ahi concatenamos toda la informacion que necesitamos.

In [ ]:
df = pd.DataFrame()

⚠️ **Alertas:** Es posible que las bases de datos no tengan exactamente las mismas columnas. Previamente preparamos un diccionario que nos va a ayudar a normalizar los nombres.

In [ ]:
columnas_ajustadas = {
    'Fuente':'Mayorista',
    'FechaEncuesta':'Fecha',
    'Fecha':'Fecha',
    'Cod. Depto Proc.':'Codigo departamento',
    'Cod. Municipio Proc.':'Codigo municipio',
    'Departamento Proc.':'Departamento',
    'Municipio Proc.':'Municipio',
    'Grupo':'Grupo',
    'Ali':'Alimento',
    'Cant Kg':'Cant Kg',
    'Código Departamento':'Codigo departamento',
    ' Código Municipio ':'Codigo municipio',
    'Alimento':'Alimento',
    'Cuidad, Mercado Mayorista':'Mayorista',
    'Origen':'Origen',
    'Unnamed: 9':'Unnamed: 9',
    'Unnamed: 10':'Unnamed: 10',
    'Codigo CPC':'Codigo CPC'
}

➡️ Ajustamos los nombres de todos los archivos de acuerdo al diccionario que acabamos de definir.

In [ ]:
for file in silver_files.iterdir():
    df_temp = pd.read_csv(file, encoding='unicode_escape', sep=";")
    df_temp['Origen'] = file.name
    for column in df_temp.columns:
        df_temp.rename(columns={column: columnas_ajustadas[column]}, inplace=True)
    df =pd.concat([df,df_temp], ignore_index=True)

➡️ Eliminamos las columnas que no vamos a utilizar.

In [ ]:
df = df.drop(['Unnamed: 9','Unnamed: 10','Codigo CPC'],axis=1)

➡️ Homologamos dentro de la columna *"Fuente"* a *"Cali, Santa Elena"* por *"Cali, Santa Helena"*



In [ ]:
df.loc[df["Mayorista"] == "Cali, Santa Elena", "Mayorista"] = "Cali, Santa Helena"

➡️ Enviamos la informacion a un archivo csv para ver como se está juntando el resultado.

In [ ]:
database_path = '/home/npalaciosv/Catedra/Geoanalitycs/src/Database/geo.db'
conn = sqlite3.connect(database_path)
df.to_sql('Fact', con=conn, if_exists='replace', index=False)

In [ ]:
def moviendo_db(database_path, user_path):
    shutil.move(database_path, user_path)
    print(f'Archivo movido a {user_path}')

In [ ]:
user_path = '/mnt/d/Classes/Data visualization/Mapping data/Database/geo.db'

moviendo_db(database_path=database_path,user_path=user_path)

### **4. Iniciamos con la construccion de los mapas**

#### **4.1 Establecemos las coordenadas de los centros mayoristas**

➡️ Primero importamos las coordenadas de cada uno de los centros de acopia que se mencionan en la base de datos

In [ ]:
coordenadas_path = "/home/npalaciosv/Catedra/Geoanalitycs/src/Data/Static/Coordenadas.csv"
df_coor = pd.read_csv(coordenadas_path,sep=";",encoding='utf-8')

conn = sqlite3.connect(user_path)
df_coor.to_sql('Coordenadas_mayoristas', con=conn, if_exists='replace', index=False)

➡️ Creamos el objeto mapa utilizando la libreria folium. Especificamos un punto central para que aparezca el mapa.

In [ ]:
map = folium.Map(location=[4.922860659232988, -74.02580517889908], zoom_start=7)

➡️ Llamamos el icono de una plaza de mercado que escogimos para ponerlo como icono de marcacion.

In [ ]:
#
icon_path = '/home/npalaciosv/Catedra/Geoanalitycs/src/Icons/Mayorista.png'

#
lat = df_coor['Latitud'].to_list()
lon = df_coor['Longitud'].to_list()
fuente = df_coor['Mayorista'].to_list()
coordenadas = list(zip(lat,lon,fuente))

#
for lat,lon,fuente in coordenadas:
    folium.Marker(
        [lat,lon],
        popup=f'{fuente}',
        icon=folium.CustomIcon(icon_image=icon_path,icon_size=(20,20))
    ).add_to(map)
map

#### **4.2 Establecemos las coordenadas de los municipios abastecedor**

In [140]:
halo = 1